In [149]:
from collections import defaultdict

with open('input.txt') as f:
    file = f.read()
droplets = []
for coor in file.split('\n'):
    coor = [int(c) for c in coor.split(',')]
    droplets.append(tuple(coor))

coor_mvmt = [(-1, 0, 0), (1, 0, 0), (0, -1, 0), (0, 1, 0), (0, 0, -1), (0, 0, 1)]

droplets_dict = defaultdict(int)
for coor in droplets:
    droplets_dict[(coor)] = 6
    x, y, z = coor
    for x_mvmt, y_mvmt, z_mvmt in coor_mvmt:
        coor_tocheck = (x + x_mvmt, y + y_mvmt, z + z_mvmt)
        # check if all values in coor to be check is more than zero
        # also check if coor is in droplets_dict
        if not all(i >= 0 for i in coor_tocheck) or coor_tocheck not in droplets_dict:
            continue
        droplets_dict[coor] -= 1
        droplets_dict[coor_tocheck] -= 1

sum(droplets_dict.values())

4310

In [150]:
# https://stackoverflow.com/questions/20229822/check-if-all-values-in-list-are-greater-than-a-certain-number
all(i > 0 for i in (-1, 0, 0))

False

In [151]:
def find_group(droplet, droplets_dict, group_lst):
    # droplet is in the form of a tuple of (x, y, z)
    x, y, z = droplet
    for x_mvmt, y_mvmt, z_mvmt in coor_mvmt:
        coor_tocheck = (x + x_mvmt, y + y_mvmt, z + z_mvmt)
        # check if droplet is already in the group list
        if not all(i >= 0 for i in coor_tocheck) or \
            coor_tocheck in group_lst or coor_tocheck not in droplets_dict:
            continue
        group_lst.append(coor_tocheck)
        find_group(coor_tocheck, droplets_dict, group_lst)


d = (2, 2, 2)
l_d = [d]
find_group(d, droplets_dict, l_d)

In [152]:
l_d

[(2, 2, 2)]

In [153]:
# part 2
# identify how many groups are there based on the current number of droplets
# actually this step is kinda useless?
checked_droplet = []
droplet_groups = []

for droplet in droplets:
    if droplet not in checked_droplet:
        d_lst = [droplet]
        find_group(droplet, droplets_dict, d_lst)
        checked_droplet.extend(d_lst)
        droplet_groups.append(d_lst)


# for each group, identify missing pockets, by calculating the min and max, or basically  coor_tocheck which are not in droplets_dict
# non_droplets do not cover all the non_droplets in entirety, we just need a catalyst to start the recursion process
non_droplets = []
for coor in droplets:
    x, y, z = coor
    for x_mvmt, y_mvmt, z_mvmt in coor_mvmt:
        coor_tocheck = (x + x_mvmt, y + y_mvmt, z + z_mvmt)
        if not all(i >= 0 for i in coor_tocheck):
            continue
        if coor_tocheck not in droplets_dict:
            non_droplets.append(coor_tocheck)

x_non_coor = [i[0] for i in non_droplets]
y_non_coor = [i[1] for i in non_droplets]
z_non_coor = [i[2] for i in non_droplets]

x_non_max, x_non_min = max(x_non_coor), min(x_non_coor)
y_non_max, y_non_min = max(y_non_coor), min(y_non_coor)
z_non_max, z_non_min = max(z_non_coor), min(z_non_coor)

# find groups of missing pockets
def find_non_group(droplet, droplets_dict, group_lst, droplet_const, droplet_i):
    if droplet_i >= droplet_const:
        return False
    # droplet is in the form of a tuple of (x, y, z)
    x, y, z = droplet
    # conclude if missing pockets reside on the edge
    # if not on the edge, then it relates to an internal droplet
    if x >= x_non_max or x <= x_non_min or y >= y_non_max or y <= y_non_min or z >= z_non_max or z <= z_non_min:
        return False

    for x_mvmt, y_mvmt, z_mvmt in coor_mvmt:
        coor_tocheck = (x + x_mvmt, y + y_mvmt, z + z_mvmt)
        # check if droplet is already in the group list
        if not all(i >= 0 for i in coor_tocheck) or \
            coor_tocheck in group_lst or coor_tocheck in droplets_dict:
            continue
        group_lst.append(coor_tocheck)
        if not find_non_group(coor_tocheck, droplets_dict, group_lst, droplet_const, droplet_i + 1):
            return False
    return True

checked_empty = []
empty_groups = []

for empty in non_droplets:
    if empty not in checked_empty:
        d_lst = [empty]
        # droplets_dict refers to valid droplets
        res = find_non_group(empty, droplets_dict, d_lst, len(droplets), 1)
        # even if res==False, extend anyway to reduce number of function calls if empty is not in checked empty
        checked_empty.extend(d_lst)
        if res:
            empty_groups.append(d_lst)


In [154]:
# basically same function as part 1
def get_surface_area(droplets_group):
    coor_mvmt = [(-1, 0, 0), (1, 0, 0), (0, -1, 0), (0, 1, 0), (0, 0, -1), (0, 0, 1)]
    droplets_dict = defaultdict(int)
    for coor in droplets_group:
        droplets_dict[(coor)] = 6
        x, y, z = coor
        for x_mvmt, y_mvmt, z_mvmt in coor_mvmt:
            coor_tocheck = (x + x_mvmt, y + y_mvmt, z + z_mvmt)
            # check if all values in coor to be check is more than zero
            # also check if coor is in droplets_dict
            if not all(i >= 0 for i in coor_tocheck) or coor_tocheck not in droplets_dict:
                continue
            droplets_dict[coor] -= 1
            droplets_dict[coor_tocheck] -= 1
    return sum(droplets_dict.values())

In [155]:
sum_l = []
for group in empty_groups:
    sum_l.append(get_surface_area(group))
print(sum(sum_l))
print(get_surface_area(droplets) - sum(sum_l))

1844
2466


In [107]:
# testing
def find_non_group(droplet, droplets_dict, group_lst, droplet_const, droplet_i):
    if droplet_i >= droplet_const:
        return False
    # droplet is in the form of a tuple of (x, y, z)
    x, y, z = droplet
    if x >= x_non_max or x <= x_non_min or y >= y_non_max or y <= y_non_min or z >= z_non_max or z <= z_non_min:
        return False

    for x_mvmt, y_mvmt, z_mvmt in coor_mvmt:
        coor_tocheck = (x + x_mvmt, y + y_mvmt, z + z_mvmt)
        # check if droplet is already in the group list
        if not all(i >= 0 for i in coor_tocheck) or \
            coor_tocheck in group_lst or coor_tocheck in droplets_dict:
            continue
        group_lst.append(coor_tocheck)
        if not find_non_group(coor_tocheck, droplets_dict, group_lst, droplet_const, droplet_i + 1):
            return False
    return True

d = (2, 2, 4)
l_ng = [d]
find_non_group(d, droplets_dict, l_ng, len(droplets), 1)

False

In [108]:
l_ng

[(2, 2, 4), (1, 2, 4), (0, 2, 4)]